In [37]:
import pandas as pd
import json
import numpy as np

In [38]:
# Read results
autoencoder_name = 'hyb3-sim_10'
results = pd.read_json('../../data/experiment-results/hybrid/%s.json' % (autoencoder_name), lines=True)

results = results.T
results['json'] = results[0]

newDataframe = pd.DataFrame(columns=['user_index', 'precision', 'recall', 'y_pred', 'y_true', 'len_yPred', 'len_yTrue', 'avg_precision', 'rmse'])
newDataframe['y_pred'] = newDataframe['y_pred'].astype(object)
newDataframe['y_true'] = newDataframe['y_true'].astype(object)

for index, row in results.iterrows():
    newSeries = pd.Series(index=['user_index', 'precision', 'recall', 'y_pred', 'y_true', 'len_yPred', 'len_yTrue'])
    newSeries['y_pred'] = newSeries['y_pred'].astype(object)
    newSeries['y_true'] = newSeries['y_true'].astype(object)

    newSeries['user_index'] = row['json']['user_index']
    newSeries['precision'] = row['json']['precision']
    newSeries['recall'] = row['json']['recall']

    things1 = np.array(row['json']['y_pred']).astype('str')
    things2 = np.array(row['json']['y_true']).astype('str')
    y_pred_string = '[' + ', '.join(things1) + ']'
    y_true_string = '[' + ', '.join(things2) + ']'
    
    newSeries['y_pred'] = y_pred_string
    newSeries['y_true'] = y_true_string
    
    newSeries['len_yPred'] = len(row['json']['y_pred'])
    newSeries['len_yTrue'] = len(row['json']['y_true'])
    
    newSeries['rmse'] = row['json']['rmse']
    newSeries['avg_precision'] = row['json']['avg_precision']
    
    newDataframe.loc[index] = newSeries
    
newDataframe.describe()

,user_index,precision,recall,avg_precision,rmse
count,896.000000,896.000000,896.000000,896.000000,8.960000e+02
mean,447.500000,0.007366,0.062109,0.004258,2.021919e-02
std,258.797218,0.030109,0.239916,0.014499,6.207327e-02
min,0.000000,0.000000,0.000000,0.000000,2.148352e-11
25%,223.750000,0.000000,0.000000,0.000000,4.373644e-11
50%,447.500000,0.000000,0.000000,0.000000,4.373644e-11
75%,671.250000,0.000000,0.000000,0.000000,4.373644e-11
max,895.000000,0.300000,1.000000,0.185185,4.303315e-01


In [39]:
print('The number of different projects suggested is %d' % (len(set(newDataframe['y_pred']))))

The number of different projects suggested is 3
